In [1]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import (
    MarketCap,
    EV,
    Fundamentals,
    Sector
)
from sharadar.pipeline.engine import load_sharadar_bundle, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
import time
import datetime
from sharadar.pipeline.universes import TRADABLE_STOCKS_US, NamedUniverse
from sharadar.pipeline.factors import Exchange, Sector, IsDomesticCommonStock, MarketCap, Fundamentals, EV, EvEbit

In [2]:
engine = make_pipeline_engine()

pipe_date = pd.to_datetime('2024-07-01', utc=False)
#pipe_date_end = pd.to_datetime('2024-02-05', utc=False)

pipe = Pipeline(columns={
    'mkt_cap': MarketCap(),
    'ev': EV(),
    'netmargin': Fundamentals(field='netmargin'),
    'revenue': Fundamentals(field='revenue'),
    'EvEbit' : EvEbit(),
    'sector': Sector(),
    }#, screen = stocks_us()
#,screen = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
,screen = NamedUniverse(TRADABLE_STOCKS_US)
#screen = TradableStocksUS()
    )

df = engine.run_pipeline(pipe, pipe_date)

[2024-07-28 21:31:24] INFO: Save root mask file: root-2024-07-01_2024-07-01_XNYS_US_0.pkl
Pipeline from 2024-07-01 to 2024-07-01
[2024-07-28 21:31:26] INFO: save term-2024-07-01_2024-07-01_screen_011b0f79315444978b8445dcfdb555c0_mkt_cap.npy to cache
[2024-07-28 21:31:26] INFO: save term-2024-07-01_2024-07-01_screen_011b0f79315444978b8445dcfdb555c0_ev.npy to cache
[2024-07-28 21:31:26] INFO: save term-2024-07-01_2024-07-01_screen_011b0f79315444978b8445dcfdb555c0_netmargin.npy to cache
[2024-07-28 21:31:26] INFO: save term-2024-07-01_2024-07-01_screen_011b0f79315444978b8445dcfdb555c0_revenue.npy to cache
[2024-07-28 21:31:26] INFO: save term-2024-07-01_2024-07-01_screen_011b0f79315444978b8445dcfdb555c0_EvEbit.npy to cache
[2024-07-28 21:31:26] INFO: save term-2024-07-01_2024-07-01_screen_011b0f79315444978b8445dcfdb555c0_sector.npy to cache
[2024-07-28 21:31:26] INFO: save term-2024-07-01_2024-07-01_screen_011b0f79315444978b8445dcfdb555c0_screen_011b0f79315444978b8445dcfdb555c0.npy to cac

In [3]:
df.head()

mkt_cap           ev  netmargin  \
2024-07-01 Equity(108199 [ATLX])  172100000.0  165000000.0    -69.432   
           Equity(108216 [AUID])   94500000.0   87500000.0    -19.428   
           Equity(108271 [MAMA])  241500000.0  241000000.0      0.019   
           Equity(108437 [BRFH])   53300000.0   52100000.0     -0.159   
           Equity(108479 [BTTR])    3300000.0    4800000.0     -0.358   

                                     revenue  EvEbit              sector  
2024-07-01 Equity(108199 [ATLX])    186707.0    -3.3     Basic Materials  
           Equity(108216 [AUID])    157378.0    -4.3          Technology  
           Equity(108271 [MAMA])  29838000.0    30.6  Consumer Defensive  
           Equity(108437 [BRFH])   2829000.0   -22.0  Consumer Defensive  
           Equity(108479 [BTTR])   7903000.0    -0.2  Consumer Defensive

In [4]:
df_group_by_sector = df.groupby('sector', observed=True)
sector_size = df_group_by_sector.size()
sector_size

sector
Basic Materials           151
Communication Services     97
Consumer Cyclical         312
Consumer Defensive        138
Energy                    150
Healthcare                396
Industrials               400
Technology                428
Utilities                  75
dtype: int64

In [5]:
# There are Sectors with size less than 30?
len(sector_size[sector_size < 30]) > 0

False

In [6]:
sector_rank = df_group_by_sector.rank(ascending=False, pct=True)
sector_rank.sort_values(by='netmargin', ascending=False).head()

mkt_cap        ev  netmargin   revenue  \
2024-07-01 Equity(631977 [JOBY])  0.427500  0.560000        1.0  1.000000   
           Equity(198778 [CDZI])  0.986667  0.986667        1.0  1.000000   
           Equity(638776 [IE])    0.668874  0.761589        1.0  0.993377   
           Equity(199020 [CKX])   0.986667  0.993333        1.0  1.000000   
           Equity(636114 [DJT])   0.350515  0.402062        1.0  1.000000   

                                    EvEbit  
2024-07-01 Equity(631977 [JOBY])  0.887500  
           Equity(198778 [CDZI])  0.973333  
           Equity(638776 [IE])    0.874172  
           Equity(199020 [CKX])   0.013423  
           Equity(636114 [DJT])   0.783505

In [7]:
score = sector_rank.mean(axis=1)
score.head()

2024-07-01  Equity(108199 [ATLX])    0.940397
            Equity(108216 [AUID])    0.892828
            Equity(108271 [MAMA])    0.689855
            Equity(108437 [BRFH])    0.952174
            Equity(108479 [BTTR])    0.954348
dtype: float64

In [8]:
score

2024-07-01  Equity(108199 [ATLX])    0.940397
            Equity(108216 [AUID])    0.892828
            Equity(108271 [MAMA])    0.689855
            Equity(108437 [BRFH])    0.952174
            Equity(108479 [BTTR])    0.954348
                                       ...   
            Equity(641912 [WAY])     0.298232
            Equity(641966 [GRAL])    0.703788
            Equity(641986 [WBTN])    0.538461
            Equity(641989 [LSH])     0.778000
            Equity(641994 [LB])      0.760000
Length: 2147, dtype: float64